In [1]:
import numpy as np
import pandas as pd
import requests
import lxml
import bs4
import html5lib
import json
import tmdbv3api

In [2]:
tmdb = tmdbv3api.TMDb()
tmdb.api_key = 'c7355ce5943fbfc6789e26e330dd164e'
movie = tmdbv3api.Movie()

In [3]:
# https://en.wikipedia.org/wiki/List_of_American_films_of_2021
pd.options.display.max_columns = None
pd.options.display.max_rows = 20

In [4]:
years_df=[None]*22
count=0
for i in range(2000,2022):
    years_df[count] = pd.read_html(io=f'http://en.wikipedia.org/wiki/List_of_American_films_of_{i}')
    count+=1

In [5]:
# 0 --2
# 1,2 --3
# 3 ---4
# 4,5,14....20 ----2,5
# 6...13 ----1,4
# 21 ---3,6

In [6]:
new_df = pd.DataFrame()

In [7]:
new_df = years_df[0][2].append(years_df[1][3].append(years_df[2][3].append(years_df[3][4],ignore_index=True)
                                                     ,ignore_index=True),ignore_index=True)

In [8]:
for i in range(0,22):
    if (i>=14 and i<=20) or (i==4) or (i==5):
        # 2 to 5
        count=2
        for j in range(count,6):
            new_df = new_df.append(years_df[i][j], ignore_index=True)
    elif (i>=6 and i<=13):
        # 1 to 4
        count=1
        for j in range(count,5):
            new_df = new_df.append(years_df[i][j], ignore_index=True)
    else:
        # 3 to 6
        count=3
        for j in range(count,7):
            new_df = new_df.append(years_df[i][j], ignore_index=True)

In [9]:
new_df = new_df[['Title','Director', 'Genre', 'Cast']]

In [10]:
new_df = new_df[~new_df['Title'].isnull()]

In [15]:
new_df['Director'] = new_df['Director'].fillna('unknown')
new_df['Genre'] = new_df['Genre'].fillna('unknown')
new_df['Cast'] = new_df['Cast'].fillna('unknown')

In [17]:
new_df.to_csv('title_handle.csv', index=False)
new_df = pd.read_csv('title_handle.csv')

In [19]:
%%time
for i in range(1,len(new_df),1):
    id = movie.search(new_df.loc[i, 'Title'])
    if id == []:
        continue
    else:
        # GETTING ID
        id = id[0].get('id')
        
        
        if new_df.loc[i, 'Director'] == 'unknown' or new_df.loc[i, 'Cast'] == 'unknown':
            result = requests.get(f'https://api.themoviedb.org/3/movie/{int(id)}/credits?api_key={tmdb.api_key}&language=en-US')
            result = result.json()
            if new_df.loc[i, 'Director'] == 'unknown':
                #Getting Director
                for r in result['crew']:
                    if r.get('department') == 'Directing':
                        new_df.loc[i, 'Director'] = r.get('name')
                        print(f"Director ----> {i}")
                        break
            if new_df.loc[i, 'Cast'] == 'unknown':
                # Getting Actor
                list_ = []
                str_ = ' '
                for r in result['cast']:
                    if len(list_)>=3:
                        break
                    else:
                        if r.get('known_for_department') == 'Acting':
                            list_.append(r.get('name'))
                new_df.loc[i, 'Cast'] = str_.join(list_)
                print(f"Actor ----> {i}")
        
        
        if new_df.loc[i, 'Genre'] == 'unknown':
            #GETTING GENRES
            genres = requests.get(url=f'https://api.themoviedb.org/3/movie/{int(id)}?api_key={tmdb.api_key}&language=en-US')
            genres = genres.json()['genres']
            list_ = []
            str_ = ' '
            for g in genres:
                list_.append(g.get('name'))
            str_ = str_.join(list_)
            if (str_ is None) or (str_ == ''):
                str_ = 'unknown'
            new_df.loc[i, 'Genre'] = str_    
            print(f"Genres ----> {i}")
        
        
        print(f"Executing ----> {i}")

Executing ----> 1
Executing ----> 2
Executing ----> 3
Executing ----> 4
Executing ----> 5
Executing ----> 6
Executing ----> 7
Executing ----> 8
Executing ----> 9
Executing ----> 10
Executing ----> 11
Executing ----> 12
Executing ----> 13
Executing ----> 14
Executing ----> 15
Executing ----> 16
Executing ----> 17
Executing ----> 18
Executing ----> 19
Executing ----> 20
Executing ----> 21
Executing ----> 22
Executing ----> 23
Executing ----> 24
Executing ----> 25
Executing ----> 26
Executing ----> 27
Executing ----> 28
Executing ----> 29
Executing ----> 30
Executing ----> 31
Executing ----> 32
Executing ----> 33
Executing ----> 34
Executing ----> 35
Executing ----> 36
Executing ----> 37
Executing ----> 38
Executing ----> 39
Executing ----> 40
Executing ----> 41
Executing ----> 42
Executing ----> 43
Executing ----> 44
Executing ----> 45
Executing ----> 46
Executing ----> 47
Executing ----> 48
Executing ----> 49
Executing ----> 50
Executing ----> 51
Executing ----> 52
Executing ----> 53
Ex

Actor ----> 330
Executing ----> 330
Actor ----> 331
Executing ----> 331
Actor ----> 332
Executing ----> 332
Actor ----> 333
Executing ----> 333
Actor ----> 334
Executing ----> 334
Actor ----> 335
Executing ----> 335
Actor ----> 336
Executing ----> 336
Actor ----> 337
Executing ----> 337
Actor ----> 338
Executing ----> 338
Actor ----> 339
Executing ----> 339
Actor ----> 340
Executing ----> 340
Actor ----> 341
Executing ----> 341
Actor ----> 342
Executing ----> 342
Actor ----> 343
Executing ----> 343
Actor ----> 344
Executing ----> 344
Actor ----> 345
Executing ----> 345
Actor ----> 346
Executing ----> 346
Actor ----> 347
Executing ----> 347
Actor ----> 348
Executing ----> 348
Actor ----> 349
Executing ----> 349
Actor ----> 350
Executing ----> 350
Actor ----> 351
Executing ----> 351
Actor ----> 352
Executing ----> 352
Actor ----> 353
Executing ----> 353
Actor ----> 354
Executing ----> 354
Actor ----> 355
Executing ----> 355
Actor ----> 356
Executing ----> 356
Actor ----> 357
Executing --

Actor ----> 559
Executing ----> 559
Actor ----> 560
Executing ----> 560
Actor ----> 561
Executing ----> 561
Actor ----> 562
Executing ----> 562
Actor ----> 563
Executing ----> 563
Actor ----> 564
Genres ----> 564
Executing ----> 564
Actor ----> 565
Executing ----> 565
Actor ----> 566
Executing ----> 566
Actor ----> 567
Executing ----> 567
Actor ----> 568
Executing ----> 568
Actor ----> 569
Executing ----> 569
Actor ----> 570
Executing ----> 570
Actor ----> 571
Executing ----> 571
Actor ----> 572
Executing ----> 572
Actor ----> 573
Executing ----> 573
Actor ----> 574
Executing ----> 574
Actor ----> 575
Executing ----> 575
Actor ----> 576
Executing ----> 576
Actor ----> 577
Executing ----> 577
Actor ----> 578
Executing ----> 578
Actor ----> 579
Executing ----> 579
Actor ----> 580
Executing ----> 580
Actor ----> 581
Executing ----> 581
Actor ----> 582
Executing ----> 582
Actor ----> 583
Executing ----> 583
Actor ----> 584
Executing ----> 584
Actor ----> 585
Executing ----> 585
Actor ---->

Actor ----> 787
Executing ----> 787
Actor ----> 788
Executing ----> 788
Actor ----> 789
Executing ----> 789
Actor ----> 790
Executing ----> 790
Actor ----> 791
Executing ----> 791
Actor ----> 792
Executing ----> 792
Actor ----> 793
Executing ----> 793
Actor ----> 794
Executing ----> 794
Actor ----> 795
Executing ----> 795
Actor ----> 796
Executing ----> 796
Actor ----> 797
Executing ----> 797
Actor ----> 798
Executing ----> 798
Actor ----> 799
Executing ----> 799
Actor ----> 800
Executing ----> 800
Actor ----> 801
Executing ----> 801
Actor ----> 802
Executing ----> 802
Actor ----> 803
Executing ----> 803
Actor ----> 804
Executing ----> 804
Actor ----> 805
Executing ----> 805
Actor ----> 806
Executing ----> 806
Actor ----> 807
Executing ----> 807
Actor ----> 808
Executing ----> 808
Actor ----> 809
Executing ----> 809
Actor ----> 810
Executing ----> 810
Actor ----> 811
Executing ----> 811
Actor ----> 812
Executing ----> 812
Actor ----> 813
Executing ----> 813
Actor ----> 814
Executing --

Actor ----> 1016
Executing ----> 1016
Actor ----> 1017
Executing ----> 1017
Actor ----> 1018
Executing ----> 1018
Actor ----> 1019
Executing ----> 1019
Actor ----> 1020
Executing ----> 1020
Actor ----> 1021
Executing ----> 1021
Actor ----> 1022
Executing ----> 1022
Actor ----> 1023
Executing ----> 1023
Actor ----> 1024
Executing ----> 1024
Actor ----> 1025
Executing ----> 1025
Actor ----> 1026
Executing ----> 1026
Actor ----> 1027
Executing ----> 1027
Actor ----> 1028
Executing ----> 1028
Actor ----> 1029
Executing ----> 1029
Actor ----> 1030
Executing ----> 1030
Actor ----> 1031
Executing ----> 1031
Actor ----> 1032
Executing ----> 1032
Actor ----> 1033
Executing ----> 1033
Actor ----> 1034
Executing ----> 1034
Actor ----> 1035
Executing ----> 1035
Actor ----> 1036
Executing ----> 1036
Actor ----> 1037
Executing ----> 1037
Actor ----> 1038
Executing ----> 1038
Actor ----> 1039
Executing ----> 1039
Actor ----> 1040
Executing ----> 1040
Actor ----> 1041
Executing ----> 1041
Actor ----> 

...

Director ----> 5760
Actor ----> 5760
Genres ----> 5760
Executing ----> 5760
Director ----> 5761
Actor ----> 5761
Genres ----> 5761
Executing ----> 5761
Director ----> 5762
Actor ----> 5762
Genres ----> 5762
Executing ----> 5762
Director ----> 5763
Actor ----> 5763
Genres ----> 5763
Executing ----> 5763
Director ----> 5764
Actor ----> 5764
Genres ----> 5764
Executing ----> 5764
Director ----> 5765
Actor ----> 5765
Genres ----> 5765
Executing ----> 5765
Director ----> 5766
Actor ----> 5766
Genres ----> 5766
Executing ----> 5766
Director ----> 5767
Actor ----> 5767
Genres ----> 5767
Executing ----> 5767
Director ----> 5768
Actor ----> 5768
Genres ----> 5768
Executing ----> 5768
Director ----> 5769
Actor ----> 5769
Genres ----> 5769
Executing ----> 5769
Director ----> 5770
Actor ----> 5770
Genres ----> 5770
Executing ----> 5770
Director ----> 5771
Actor ----> 5771
Genres ----> 5771
Executing ----> 5771
Director ----> 5772
Actor ----> 5772
Genres ----> 5772
Executing ----> 5772
Director ---

Genres ----> 5868
Executing ----> 5868
Director ----> 5869
Actor ----> 5869
Genres ----> 5869
Executing ----> 5869
Director ----> 5870
Actor ----> 5870
Genres ----> 5870
Executing ----> 5870
Director ----> 5872
Actor ----> 5872
Genres ----> 5872
Executing ----> 5872
Director ----> 5873
Actor ----> 5873
Genres ----> 5873
Executing ----> 5873
Director ----> 5874
Actor ----> 5874
Genres ----> 5874
Executing ----> 5874
Director ----> 5875
Actor ----> 5875
Genres ----> 5875
Executing ----> 5875
Director ----> 5876
Actor ----> 5876
Genres ----> 5876
Executing ----> 5876
Director ----> 5877
Actor ----> 5877
Genres ----> 5877
Executing ----> 5877
Director ----> 5878
Actor ----> 5878
Genres ----> 5878
Executing ----> 5878
Director ----> 5879
Actor ----> 5879
Genres ----> 5879
Executing ----> 5879
Director ----> 5880
Actor ----> 5880
Genres ----> 5880
Executing ----> 5880
Director ----> 5881
Actor ----> 5881
Genres ----> 5881
Executing ----> 5881
Director ----> 5882
Actor ----> 5882
Genres ---->

Genres ----> 5977
Executing ----> 5977
Director ----> 5978
Actor ----> 5978
Genres ----> 5978
Executing ----> 5978
Director ----> 5979
Actor ----> 5979
Genres ----> 5979
Executing ----> 5979
Director ----> 5980
Actor ----> 5980
Genres ----> 5980
Executing ----> 5980
Director ----> 5981
Actor ----> 5981
Genres ----> 5981
Executing ----> 5981
Director ----> 5982
Actor ----> 5982
Genres ----> 5982
Executing ----> 5982
Director ----> 5983
Actor ----> 5983
Genres ----> 5983
Executing ----> 5983
Director ----> 5984
Actor ----> 5984
Genres ----> 5984
Executing ----> 5984
Director ----> 5985
Actor ----> 5985
Genres ----> 5985
Executing ----> 5985
Director ----> 5986
Actor ----> 5986
Genres ----> 5986
Executing ----> 5986
Director ----> 5987
Actor ----> 5987
Genres ----> 5987
Executing ----> 5987
Director ----> 5988
Actor ----> 5988
Genres ----> 5988
Executing ----> 5988
Director ----> 5989
Actor ----> 5989
Genres ----> 5989
Executing ----> 5989
Director ----> 5990
Actor ----> 5990
Genres ---->

Genres ----> 6085
Executing ----> 6085
Director ----> 6086
Actor ----> 6086
Genres ----> 6086
Executing ----> 6086
Director ----> 6087
Actor ----> 6087
Genres ----> 6087
Executing ----> 6087
Director ----> 6088
Actor ----> 6088
Genres ----> 6088
Executing ----> 6088
Director ----> 6089
Actor ----> 6089
Genres ----> 6089
Executing ----> 6089
Director ----> 6090
Actor ----> 6090
Genres ----> 6090
Executing ----> 6090
Director ----> 6091
Actor ----> 6091
Genres ----> 6091
Executing ----> 6091
Director ----> 6092
Actor ----> 6092
Genres ----> 6092
Executing ----> 6092
Director ----> 6093
Actor ----> 6093
Genres ----> 6093
Executing ----> 6093
Director ----> 6094
Actor ----> 6094
Genres ----> 6094
Executing ----> 6094
Director ----> 6095
Actor ----> 6095
Genres ----> 6095
Executing ----> 6095
Director ----> 6096
Actor ----> 6096
Genres ----> 6096
Executing ----> 6096
Director ----> 6097
Actor ----> 6097
Genres ----> 6097
Executing ----> 6097
Director ----> 6098
Actor ----> 6098
Genres ---->

Director ----> 6194
Actor ----> 6194
Genres ----> 6194
Executing ----> 6194
Director ----> 6195
Actor ----> 6195
Genres ----> 6195
Executing ----> 6195
Director ----> 6196
Actor ----> 6196
Genres ----> 6196
Executing ----> 6196
Director ----> 6197
Actor ----> 6197
Genres ----> 6197
Executing ----> 6197
Director ----> 6198
Actor ----> 6198
Genres ----> 6198
Executing ----> 6198
Director ----> 6199
Actor ----> 6199
Genres ----> 6199
Executing ----> 6199
Director ----> 6200
Actor ----> 6200
Genres ----> 6200
Executing ----> 6200
Director ----> 6201
Actor ----> 6201
Genres ----> 6201
Executing ----> 6201
Director ----> 6202
Actor ----> 6202
Genres ----> 6202
Executing ----> 6202
Director ----> 6203
Actor ----> 6203
Genres ----> 6203
Executing ----> 6203
Director ----> 6204
Actor ----> 6204
Genres ----> 6204
Executing ----> 6204
Director ----> 6205
Actor ----> 6205
Genres ----> 6205
Executing ----> 6205
Director ----> 6206
Actor ----> 6206
Genres ----> 6206
Executing ----> 6206
Director ---

In [20]:
new_df.to_csv('saving.csv', index=False)

In [13]:
new_df = pd.read_csv('saving.csv')

In [14]:
new_df = new_df.fillna('unknown')

In [15]:
for i in range(len(new_df)):
    new_df.loc[i, 'Cast'] = new_df.loc[i, 'Cast'].replace(',', ' ')
    new_df.loc[i, 'Genre'] = new_df.loc[i, 'Genre'].replace(',', ' ')

In [16]:
#  ‡
for i in range(len(new_df)):
    if '‡' in new_df.loc[i, 'Title']:
        new_df.loc[i, 'Title'] = new_df.loc[i, 'Genre'].replace('‡', '')

In [17]:
new_df = new_df.drop_duplicates(subset='Title', keep='first')

In [18]:
for col in new_df.columns:
    new_df[col] = new_df[col].str.lower()

In [19]:
new_df['combined'] = new_df['Title'] + ' ' +new_df['Director'] + ' ' + new_df['Genre'] + ' ' + new_df['Cast']

In [20]:
new_df.to_csv('pre-processed.csv', index=False)